# Dirichlet boundary conditions
We are solving the Poisson equation 

$$\text{find: } u \in H^1 \cap \{ u|_{\Gamma_D} = g_D\} \text{ s.t.} \quad \int_\Omega \nabla u \nabla v = \int_\Omega f v \quad \forall v \in H_{0,D}^1 = H^1 \cap \{ u|_{\Gamma_D} = 0 \} $$

We do this by homogenizing, i.e. we specify a function $u_D \in H^1$ with $u_D|_{\Gamma_D} = g_D$ and solve for $u_0 \in H_{0,D}^1$, so that $u = u_0 + u_D$. So, we ask the questions:
 * How to define (approximate) a function $u_D$?
 * How to solve for $u_0$?
We have already seen that the geometry has names and indices for different parts in the boundary that we can use to specify **dirichlet**-parts of the boundary for a finite element space:

In [ ]:
import netgen.gui
%gui tk
from ngsolve import *
from netgen.geom2d import unit_square
mesh = Mesh(unit_square.GenerateMesh(maxh=0.2))
fes = H1(mesh, order=2, dirichlet="left|right")
print(H1(mesh, order=2, dirichlet="").ndof)
print(fes.ndof)

We note: the dirichlet-flag does not change the number of unknowns 

In NGSolve the unknowns are split into two groups: 
 * dirichlet-dofs and 
 * free dofs
First, matrices and vectors are set up w.r.t. all dofs. Based on an additional field (a BitArray with values True or False for every dof) the splitting has to respected when solving the linear systems.

In [ ]:
print("free dofs of fes without \"dirichlet\" flag:\n", H1(mesh, order=2, dirichlet="").FreeDofs())
print("free dofs of fes:\n", fes.FreeDofs())

We observe that the space without dirichlet-flag has only free dofs (no dirichlet dof) while the other space has a few dofs that are marked as non-free dofs. These are the dofs that are located on surface elements which are on the part of the surface that has the corresponding Dirichlet identifier (index or matching regexp-string). 

Bilinear and linear forms are also defined independent of the dirichlet flag:

In [ ]:
u = fes.TrialFunction()
v = fes.TestFunction()

a = BilinearForm(fes, symmetric=True)
a += SymbolicBFI (grad(u)*grad(v))
a.Assemble()

f = LinearForm(fes)
f += SymbolicLFI(1*v)
f.Assemble()

To define $u_D$ we can use the *Set* method which implements a Oswald-type interpolation ($L^2$-projection on each element + averaging). Set can be used in (parts of) the domain / boundary. Note that when ever you use set the vector is set to zero before.

In [ ]:
gfu = GridFunction(fes)
gfu.Set(CoefficientFunction(1),definedon = mesh.Boundaries("left"))
gfu.Set(CoefficientFunction(1),definedon = mesh.Boundaries("right"))
gfu.Set(CoefficientFunction(1),definedon = mesh.Boundaries("top"))
gfu.Set(CoefficientFunction(1),definedon = mesh.Boundaries("bottom"))
Draw(gfu,mesh,"u")

Here setting values on one part of the boundary overwrote the others. To set boundary value $1$ on the left and right boundary we use the regular expression "left|right":

In [ ]:
gfu = GridFunction(fes)
gfu.Set(CoefficientFunction(1),definedon = mesh.Boundaries("left|right"))
Draw(gfu,mesh,"u")

We now want to solve 
$$
  A (u_0 + u_D) = f \quad \Rightarrow \quad A u_0 = f - A u_D
$$
Actually, we want to solve only for the equations of $A u_0 = f - A u_D$ corresponding to free dofs. If we think of free dofs and dirichlet dofs numbered consecutively (which is typically not true), we have
$$
  \left( \begin{array}{cc} A_{FF} & A_{FD} \\ A_{DF} & A_{DD} \end{array} \right) \left( \begin{array}{c} u_{F} \\ u_{D} \end{array} \right) = \left( \begin{array}{c} {f}_F \\ {f}_D \end{array} \right) \quad \begin{array}{l} \leftarrow \text{ solved for } u_{F} \\ \leftarrow \text{ replaced with } u_{D} = g_D\end{array}
$$
which becomes
$$
  \left( \begin{array}{cc} A_{FF} & A_{FD} \\ A_{DF} & A_{DD} \end{array} \right) \left( \begin{array}{c} u_{0,F} \\ u_{0,D} \end{array} \right) = \left( \begin{array}{c} {f}_F \\ {f}_D \end{array} \right)
%  
 - \left( \begin{array}{cc} A_{FF} & A_{FD} \\ A_{DF} & A_{DD} \end{array} \right) \left( \begin{array}{c} u_{D,F} \\ u_{D,D} \end{array} \right)
% \begin{array}{l} \leftarrow \text{ solved for } u_{0,F} \\ \leftarrow \text{ replaced with } u_{0,D}  
%  \quad \begin{array}{l} \leftarrow \text{ solved for } u_{0,F} \\ \leftarrow \text{ replaced with } u_{0,D} = g_D\end{array}
$$
and with $u_{0,D} = 0$ and the second row replaced with the prescribed boundary data:
$$
  \left( \begin{array}{cc} A_{FF} &  \\  &  \end{array} \right) \left( \begin{array}{c} u_{0,F} \\ {}  \end{array} \right) = \left( \begin{array}{c} {f}_F \\ {} \end{array} \right)
%  
 - \left( \begin{array}{cc} A_{FF} & A_{FD} \\  &  \end{array} \right) \left( \begin{array}{c} u_{D,F} \\ u_{D,D} \end{array} \right) =  \tilde{f}
$$
Hence, we need to
* set up the right hand side (from a given $f$ and a given $u_D$)
* solve a linear system which involves $A_{FF}$ only
* add solution: $u = u_0 + u_D$


Setting up the right hand side (from a given $f$ and a given $u_D$) is easy:

In [ ]:
f.vec.data -= a.mat * gfu.vec

Note that this affects all entries of $\tilde{f}$, although we are only interested in the parts corresponding to free dofs, $\tilde{f}_F$.

To solve for the $A_{FF}$ block we can use sparse direct solvers on $A$ with the freedofs-argument which prescribes the subblock that is to be inverted, i.e. 
$$
 A.Inverse(freedofs=...) = \left( \begin{array}{cc} A_{FF}^{-1} & 0 \\ 0 & 0 \end{array} \right)
$$
With
$$
  u = u_0 + \left( \begin{array}{cc} A_{FF}^{-1} & 0 \\ 0 & 0 \end{array} \right) \cdot \tilde{f}
$$
we simply have

In [ ]:
gfu.vec.data += a.mat.Inverse(freedofs=fes.FreeDofs()) * f.vec
Redraw()

## Using BVP

Using BVP homogenization is taken care of automatically. You provide $A$, $f$, $u$ and a preconditioner and the problem is solved for. Here, the preconditioner only acts on the free dofs:

In [ ]:
c = Preconditioner(a,"local")     #<- Jacobi preconditioner
#c = Preconditioner(a,"direct")   #<- sparse direct solver
c.Update()
BVP(bf=a,lf=f,gf=gfu,pre=c).Do()
Redraw()